In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time
def web_scraping_bnr(url:str,service:str) -> pd.DataFrame:
    # Configurar Selenium (asegúrate de tener ChromeDriver instalado)
    service = Service(service)  # Cambia esta ruta a la ubicación de tu chromedriver
    driver = webdriver.Chrome(service=service)

    # URL de la página
    url_bnr = url
    driver.get(url_bnr)
    time.sleep(15)
    # Esperar a que la página cargue (puedes ajustar según sea necesario)
    driver.implicitly_wait(10)

    # Obtener el HTML de la página después de cargar
    html = driver.page_source
    driver.quit()

    # Parsear el contenido HTML con Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')

    # Buscar la tabla específica usando su clase
    table = soup.find('table', class_="table-color max-h table")

    # Extraer los encabezados (que están todos en la fila <tr> principal con <th>)
    headers = [header.text.strip() for header in table.find('tr').find_all('th')]

    # Extraer las filas de datos
    rows = []
    for row in table.find_all('tr')[1:]:  # Excluir la primera fila que contiene los encabezados
        cells = row.find_all(['th', 'td'])  # Incluye el <th> del mes y los <td> de las celdas
        row_data = [cell.text.strip() for cell in cells]
        rows.append(row_data)

    # Crear un DataFrame con los encabezados y las filas extraídas
    df = pd.DataFrame(rows, columns=headers)

    # Guardar los datos en un archivo CSV con el formato esperado
    df.to_csv('/home/host/airflow_prueba/analitica/pruebitas_git/monthly_series_averages.csv', index=False)
    print("Datos extraídos y guardados en '/home/host/airflow_prueba/analitica/pruebitas_git/monthly_series_averages.csv'.")
    return df


In [24]:
#df = web_scraping_bnr("https://www.bnr.ro/en/23793-exchange-rates-monthly-quarterly-and-annual-averages","/usr/bin/chromedriver")
df = web_scraping_bnr("https://www.bnr.ro/en/23793-exchange-rates-monthly-quarterly-and-annual-averages","/usr/bin/chromedriver")
df[["Month","EUR","USD"]].head(1)

KeyboardInterrupt: 

In [82]:
import requests
from bs4 import BeautifulSoup

def web_scraping_bnr():
    # URL del endpoint
    url = 'https://www.bnr.ro//blocks'

    # Payload encontrado en Network -> Payload
    payload = {
        'bid': '26412',
        'currentSlug': '23793-exchange-rates-monthly-quarterly-and-annual-averages',
        'cat_id': ''
    }

    # Cabeceras HTTP necesarias
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': 'https://www.bnr.ro/en/23793-exchange-rates-monthly-quarterly-and-annual-averages',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
    }

    # Hacer la petición POST
    response = requests.post(url, data=payload, headers=headers)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        # Encontrar la tabla
        table = soup.find('table')
        if table:
            # Extraer encabezados
            headers = [h.get_text(strip=True) for h in table.find('thead').find_all('th')]

            # Extraer filas
            rows_data = []
            for row in table.find('tbody').find_all('tr'):
                # La primera celda es un <th> con la fecha
                month = row.find('th').get_text(strip=True)
                # El resto son <td>
                cells = row.find_all('td')
                data = [cell.get_text(strip=True) for cell in cells]
                # Combinar
                row_values = [month] + data
                rows_data.append(row_values)

            # Combinar cabeceras y filas en una lista
            full_table = [headers] + rows_data

            # Retornar la lista completa
            return full_table[:2]
        else:
            return "No se encontró la tabla en la respuesta."
    else:
        return f"La petición falló con el código de estado {response.status_code}."


[['Month', 'AUD', 'BGN', 'CAD', 'CHF', 'CZK', 'DKK', 'EGP', 'EUR', 'GBP', '100 HUF', '100 JPY', 'MDL', 'NOK', 'PLN', 'RUB', 'SEK', 'TRY', 'USD', 'ZAR', 'BRL', 'CNY', 'INR', '100 KRW', 'MXN', 'NZD', 'RSD', 'UAH', 'AED', 'THB', 'HKD', 'ILS', '100 IDR', 'PHP', '100 ISK', 'MYR', 'SGD', 'XAU', 'XDR'], ['11/2024', '3.0587', '2.5442', '3.3512', '5.3189', '0.1966', '0.6672', '0.0948', '4.9760', '5.9666', '1.2148', '3.0489', '0.2589', '0.4237', '1.1475', '0.0464', '0.4296', '0.1359', '4.6834', '0.2612', '0.8084', '0.6496', '0.0555', '0.3357', '0.2304', '2.7693', '0.0425', '0.1131', '1.2751', '0.1360', '0.6020', '1.2596', '0.0295', '0.0797', '3.3926', '1.0560', '3.5031', '399.8943', '6.1744']]


In [88]:
xd = pd.DataFrame(full_table[1:3], columns = full_table[0])
xd

,Month,AUD,BGN,CAD,CHF,CZK,DKK,EGP,EUR,GBP,...,THB,HKD,ILS,100 IDR,PHP,100 ISK,MYR,SGD,XAU,XDR
0,11/2024,3.0587,2.5442,3.3512,5.3189,0.1966,0.6672,0.0948,4.9760,5.9666,...,0.1360,0.6020,1.2596,0.0295,0.0797,3.3926,1.0560,3.5031,399.8943,6.1744
1,10/2024,3.0613,2.5435,3.3180,5.3004,0.1967,0.6669,0.0939,4.9748,5.9564,...,0.1366,0.5870,1.2129,0.0294,0.0794,3.3356,1.0611,3.4828,394.5996,6.0963


In [77]:
web_scraping_bnr()

39
39
39
39
39
39
39
39
39
39
39
39
39
None


[['Month', 'EUR', 'USD']]

In [11]:
from datetime import datetime, timedelta

XD = datetime.strptime("2024-10-11", "%Y-%m-%d").year,datetime.strptime("2024-10-11", "%Y-%m-%d").month
print(type(XD),XD)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [12]:
import requests
from bs4 import BeautifulSoup

def web_scraping_bnr() -> any:
    # URL del endpoint
    url = 'https://www.bnr.ro//blocks'

    # Payload encontrado en Network -> Payload
    payload = {
        'bid': '26412',
        'currentSlug': '23793-exchange-rates-monthly-quarterly-and-annual-averages',
        'cat_id': ''
    }

    # Cabeceras HTTP necesarias
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': 'https://www.bnr.ro/en/23793-exchange-rates-monthly-quarterly-and-annual-averages',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
    }

    # Hacer la petición POST
    response = requests.post(url, data=payload, headers=headers)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        # Encontrar la tabla
        table = soup.find('table')
        if table:
            # Extraer encabezados
            headers = [h.get_text(strip=True) for h in table.find('thead').find_all('th')]

            # Extraer filas
            rows_data = []
            for row in table.find('tbody').find_all('tr'):
                # La primera celda es un <th> con la fecha
                month = row.find('th').get_text(strip=True)
                # El resto son <td>
                cells = row.find_all('td')
                data = [cell.get_text(strip=True) for cell in cells]
                # Combinar
                row_values = [month] + data
                rows_data.append(row_values)

            # Combinar cabeceras y filas en una lista
            full_table = [headers] + rows_data

            # Retornar la lista completa
            return full_table[:3]
        else:
            return "No se encontró la tabla en la respuesta."
    else:
        return f"La petición falló con el código de estado {response.status_code}."


In [19]:
import pandas as pd 
xd = web_scraping_bnr()
xd2 = pd.DataFrame(xd)
xd2.columns = xd2.iloc[0]
xd2 = xd2[1:].reset_index(drop=True)

In [22]:
xd2[["Month","EUR","USD"]][xd2[["Month","EUR","USD"]]["Month"]=="10/2024"]

,Month,EUR,USD
1,10/2024,4.9748,4.5615
